# CHAT INFERENCE MODE


In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

/home/zhaoyang/.conda/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
base_model_name = "/home/zhaoyang/Documents/hf_models/meta-llama/Llama-2-13b-chat-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)

base_model.config.use_cache = False

base_model.config.pretraining_tp = 1 


/home/zhaoyang/.conda/envs/py310/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
dialogs = """\
[INST] <>
You are a helpful, respectful and honest sentiment analysis assistant. Analyze two sentences for semantic inversion. Consider the adversarial sentence, which employs word substitution leading to dissimilar meanings. Assess if the change maintains legal and logical consistency. Provide a binary response of the user's message into one of the following categories: legal or illegal.
<>

Here is an inference example:
Sentence: Original->The cat sat on the mat. Adversarial->The cat sat on the hat.
Sentiment: [/INST] illegal [INST] Here is another inference example:
Sentence: Original->He quickly finished his homework. Adversarial->He quickly completed his homework.
Sentiment: [/INST] legal [INST] Please classify the sentiment of the following sentence into legal or illegal:
Sentence: Original->this goofy gangster yarn never really elevates itself from being yet another earnestly generic crime busting comic vehicle a well intentioned remake that shows some spunk and promise but fails to register as anything distinctive or daring. Adversarial->this goofy gangster yarn never really elevates itself from being yet another earnestly generic crime busting comic vehicle a well intentioned remake that shows some spunk and promise but overlooking to register as everythings distinctive or daring.
Sentiment: [/INST] 
"""

In [ ]:

inputs = tokenizer(dialogs, return_tensors="pt").to("cuda")
outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

[INST] <>
You are a helpful, respectful and honest sentiment analysis assistant. Analyze two sentences for semantic inversion. Consider the adversarial sentence, which employs word substitution leading to dissimilar meanings. Assess if the change maintains legal and logical consistency. Provide a binary response of the user's message into one of the following categories: legal or illegal.
<>

Here is an inference example:
Sentence: Original->The cat sat on the mat. Adversarial->The cat sat on the hat.
Sentiment: [/INST] illegal [INST] Here is another inference example:
Sentence: Original->He quickly finished his homework. Adversarial->He quickly completed his homework.
Sentiment: [/INST] legal [INST] Please classify the sentiment of the following sentence into legal or illegal:
Sentence: Original->this goofy gangster yarn never really elevates itself from being yet another earnestly generic crime busting comic vehicle a well intentioned remake that shows some spunk and promise but fail

# Test Inference Mode 1

In [ ]:
system_prompt = "You are a helpful, respectful and honest sentiment analysis assistant. Analyze two sentences for semantic inversion. Consider the adversarial sentence, which employs word substitution leading to dissimilar meanings. Assess if the change maintains legal and logical consistency. Provide a binary response of the user's message into one of the following categories: legal or illegal."
sentence = "Original->this goofy gangster yarn never really elevates itself from being yet another earnestly generic crime busting comic vehicle a well intentioned remake that shows some spunk and promise but fails to register as anything distinctive or daring. Adversarial->this goofy gangster yarn never really elevates itself from being yet another earnestly generic crime busting comic vehicle a well intentioned remake that shows some spunk and promise but overlooking to register as everythings distinctive or daring."
examples = """
Here is an inference example:
Sentence: Original->The cat sat on the mat. Adversarial->The cat sat on the hat.
Sentiment: illegal

Here is another inference example:
Sentence: Original->He quickly finished his homework. Adversarial->He quickly completed his homework.
Sentiment: legal
"""

# text = system_prompt + "\n" + user_msg_1 + "\n" + "Sentiment:"
text = system_prompt + examples + "\n" + "Sentence: " + sentence + "\n" + "Sentiment: "

inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = base_model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=False))

<s> You are a helpful, respectful and honest sentiment analysis assistant. Analyze two sentences for semantic inversion. Consider the adversarial sentence, which employs word substitution leading to dissimilar meanings. Assess if the change maintains legal and logical consistency. Provide a binary response of the user's message into one of the following categories: legal or illegal.
Here is an inference example:
Sentence: Original->The cat sat on the mat. Adversarial->The cat sat on the hat.
Sentiment: illegal

Here is another inference example:
Sentence: Original->He quickly finished his homework. Adversarial->He quickly completed his homework.
Sentiment: legal

Sentence: Original->this goofy gangster yarn never really elevates itself from being yet another earnestly generic crime busting comic vehicle a well intentioned remake that shows some spunk and promise but fails to register as anything distinctive or daring. Adversarial->this goofy gangster yarn never really elevates itself f

# Inference Mode 2


In [ ]:
system_prompt = "You are a helpful, respectful and honest sentiment analysis assistant. Analyze two sentences for semantic inversion. Consider the adversarial sentence, which employs word substitution leading to dissimilar meanings. Assess if the change maintains legal and logical consistency. Provide a binary response of the user's message into one of the following categories: legal or illegal."
sentence = "Original->this goofy gangster yarn never really elevates itself from being yet another earnestly generic crime busting comic vehicle a well intentioned remake that shows some spunk and promise but fails to register as anything distinctive or daring. Adversarial->this goofy gangster yarn never really elevates itself from being yet another earnestly generic crime busting comic vehicle a well intentioned remake that shows some spunk and promise but overlooking to register as everythings distinctive or daring."

# text = system_prompt + "\n" + user_msg_1 + "\n" + "Sentiment:"
text = system_prompt + examples + "\n" + "Sentence: " + sentence + "\n" + "Sentiment: "

pipe = pipeline(task="text-generation", model=base_model, tokenizer=tokenizer, max_length=100)
result = pipe(f"{text}")
print(result[0]['generated_text'])

You are a helpful, respectful and honest sentiment analysis assistant. Analyze two sentences for semantic inversion. Consider the adversarial sentence, which employs word substitution leading to dissimilar meanings. Assess if the change maintains legal and logical consistency. Provide a binary response of the user's message into one of the following categories: legal or illegal.
Here is an inference example:
Sentence: Original->The cat sat on the mat. Adversarial->The cat sat on the hat.
Sentiment: illegal

Here is another inference example:
Sentence: Original->He quickly finished his homework. Adversarial->He quickly completed his homework.
Sentiment: legal

Sentence: Original->this goofy gangster yarn never really elevates itself from being yet another earnestly generic crime busting comic vehicle a well intentioned remake that shows some spunk and promise but fails to register as anything distinctive or daring. Adversarial->this goofy gangster yarn never really elevates itself from 

/home/zhaoyang/.conda/envs/py310/lib/python3.10/site-packages/transformers/generation/utils.py:1281: UserWarning: Input length of input_ids is 279, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(
